In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sportypy

player_data = pd.read_pickle("pickled_data/player_info")
plays_data = pd.read_pickle("pickled_data/game_plays")
plays_players_data = pd.read_pickle("pickled_data/game_plays_players")
player_info = pd.read_pickle("pickled_data/player_info")

player_info['birthDate'] = pd.to_datetime(player_info['birthDate'], errors='coerce')

merged_df = plays_data.merge(plays_players_data, on='play_id', how='left')
merged_df = merged_df.merge(player_info, left_on='player_id', right_on='player_id', how='left')
current_year = 2020
merged_df = merged_df[(current_year - merged_df['birthDate'].dt.year >= 20) & (current_year - merged_df['birthDate'].dt.year <= 30)]

In [25]:
scoring_plays = ['Goal', 'Assist']
defensive_plays = ['Blocked Shot', 'Takeaway', 'Goal', 'Assist']

offensive_positions = ['C', 'LW', 'RW']
defensive_positions = ['D']

offensive_df = merged_df[merged_df['event'].isin(scoring_plays)]
defensive_df = merged_df[merged_df['event'].isin(defensive_plays)]

offensive_df = merged_df[merged_df['primaryPosition'].isin(offensive_positions)]
defensive_df = merged_df[merged_df['primaryPosition'].isin(defensive_positions)]

offensive_stats = offensive_df.groupby('player_id').size().reset_index(name='offensive_events')
defensive_stats = defensive_df.groupby('player_id').size().reset_index(name='defensive_events')

offensive_stats = offensive_stats.merge(player_info, on='player_id', how='left')
defensive_stats = defensive_stats.merge(player_info, on='player_id', how='left')

low_offensive, high_offensive = offensive_stats['offensive_events'].quantile([0.15, 0.65])
low_defensive, high_defensive = defensive_stats['defensive_events'].quantile([0.15, 0.65])

offensive_stats = offensive_stats[(offensive_stats['offensive_events'] >= low_offensive) & (offensive_stats['offensive_events'] <= high_offensive)]
defensive_stats = defensive_stats[(defensive_stats['defensive_events'] >= low_defensive) & (defensive_stats['defensive_events'] <= high_defensive)]



In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(offensive_stats['offensive_events'], bins = 30, kde = True)
plt.title("Distribution of Offensive Production")
plt.xlabel("Number of Offensive Events")
plt.ylabel("Frequency")
plt.show()

plt.figure(figsize=(10,5))
sns.histplot(defensive_stats['defensive_events'], bins = 30, kde = True)
plt.title("Distribution of Defensive Production")
plt.xlabel("Number of Defensive Events")
plt.ylabel("Frequency")
plt.show()

offensive_mean = offensive_stats['offensive_events'].mean()
offensive_std = offensive_stats['offensive_events'].std()
league_avg_offensive = offensive_stats[(offensive_stats['offensive_events'] <= offensive_mean + offensive_std) & (offensive_stats['offensive_events'] >= offensive_mean)]

defensive_mean = defensive_stats['defensive_events'].mean()
defensive_std = defensive_stats['defensive_events'].std()
league_avg_defensive = defensive_stats[(defensive_stats['defensive_events'] <= defensive_mean + defensive_std) & (defensive_stats['defensive_events'] >= defensive_mean)]

random_offensive_sample = league_avg_offensive.sample(n=min(10, len(league_avg_offensive)), random_state=42)
random_defensive_sample = league_avg_defensive.sample(n=min(10,len(league_avg_defensive)), random_state=42)

display(random_defensive_sample)


In [37]:
from matplotlib.patches import Arc, Wedge
def draw_rink(ax):
    '''
    Displays a hockey rink with NHL dimensions at current axes
    Parameter ax: Current axes
    Returns nothing
    '''

    # Draw the center ice line
    ax.axvline(0, color='red', linestyle='-', linewidth=2, alpha=0.5)
    
    # Draw the blue lines at +/- 25 feet from center ice
    ax.axvline(-25, color='blue', linestyle='--', linewidth=2, alpha=0.5)
    ax.axvline(25, color='blue', linestyle='--', linewidth=2, alpha=0.5)

    # Draw blue faceoff circle at center ice (15 ft radius) with blue dot at the center
    center_circle = plt.Circle((0, 0), 15, edgecolor='blue', facecolor='none', alpha = 0.5, lw=2)
    ax.add_patch(center_circle)
    ax.plot(0, 0, marker='o', color='blue', alpha = 0.5, markersize=6)  
    
    # Draw the 4 red faceoff circles with red dots at center (31 feet from end boards and 20.5 feet from side boards)
    faceoff_positions = [(69, 20.5), (-69, 20.5), (69, -20.5), (-69, -20.5)]
    for x, y in faceoff_positions:
        faceoff_circle = plt.Circle((x, y), 15, edgecolor='red', facecolor='none', alpha = 0.5, lw=2)
        ax.add_patch(faceoff_circle)
        ax.plot(x, y, marker='o', color='red', alpha = 0.5, markersize=6)  
    
    # Draw goal lines for net (6 ft)
    ax.plot([-89, -89], [-3, 3], color='red', lw=1)  # Left goal line
    ax.plot([89, 89], [-3, 3], color='red', lw=1)    # Right goal line

    # Draw back of goals as arcs (6 ft wide and 4 ft deep)
    left_goal= Arc((-89, 0), width=6, height=8, angle=90, theta1=360, theta2=180, color='red', lw=1)
    right_goal = Arc((89, 0), width=6, height=8, angle=90, theta1=180, theta2=360, color='red', lw=1)

    ax.add_patch(left_goal)
    ax.add_patch(right_goal)

    # Draw the goal creases using Wedges and fill with low opacity (6 ft radius)
    left_goal_crease = Wedge((-89, 0), r=6, theta1=270, theta2=90, color='skyblue', alpha=0.2, edgecolor='blue', lw=2)
    right_goal_crease = Wedge((89, 0), r=6, theta1=90, theta2=270, color='skyblue', alpha=0.2, edgecolor='blue', lw=2)

    ax.add_patch(left_goal_crease)
    ax.add_patch(right_goal_crease)

    # Set the rink bounds (200 ft by 85 ft)
    ax.set_xlim(-100, 100)        
    ax.set_ylim(-42.5, 42.5)

    # Treat x and y units equally so circles are drawn correctly
    ax.set_aspect('equal')

In [ ]:
sampled_offplayer_ids = random_offensive_sample['player_id'].tolist()
filtered_offplays = merged_df[(merged_df['player_id'].isin(sampled_offplayer_ids)) & (merged_df['event'].isin(scoring_plays))]


fig, axes = plt.subplots(2, 5, figsize=(24,12))
axes = axes.flatten()

for i, (player_id) in enumerate(sampled_offplayer_ids):
    ax = axes[i]
    player_shots_goals = filtered_offplays[filtered_offplays['player_id'] == player_id]

    draw_rink(ax)

    sns.scatterplot(x=player_shots_goals['st_x'], y=player_shots_goals['st_y'], hue=player_shots_goals['secondaryType'], s=80, alpha=0.7, ax=ax)
    ax.legend(loc='upper left')
    
    player_name = filtered_offplays[filtered_offplays['player_id'] == player_id]['fullName'].iloc[1]
    ax.set_title(f"{player_name} Goal Chart", fontsize=14)
    

plt.show()

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(24,16))

axes = axes.flatten()

for i, (player_id) in enumerate(sampled_offplayer_ids):
    ax = axes[i]
    player_shots_goals = filtered_offplays[filtered_offplays['player_id'] == player_id]

    draw_rink(ax)

    sns.kdeplot(x='st_x', y='st_y', data=player_shots_goals, thresh=0.2, fill=True, cmap='coolwarm', ax=ax)
    player_name = filtered_offplays[filtered_offplays['player_id'] == player_id]['fullName'].iloc[1]
    ax.set_title(f"{player_name} Goal Chart", fontsize=14)

plt.show()
    
    